# Welcome to my notebook!

Default code from Kaggle Notebook:

In [18]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
   
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

   

## Displaying some important libraries:

In [19]:
import tensorflow as tf
print("Tensorflow:", tf.__version__)

import kerastuner as kt
print("kerastuner:", kt.__version__)

import keras_tuner as kt2
print("keras_tuner:", kt2.__version__)

import platform
print("Python:", platform.python_version())

import numpy as np
print("numpy:", np.__version__)

import pandas as pd
print("pandas:", pd.__version__)

import sklearn
print("sklearn version:", sklearn.__version__)

import sklearn
print("sklearn path:", sklearn.__path__)

import matplotlib
print("matplotlib:", matplotlib.__version__)

import seaborn as sns
print("seaborn:", sns.__version__)

# On WSL

# 2024-01-30 11:17:52.768682: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
# 2024-01-30 11:17:53.149956: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
# 2024-01-30 11:17:53.150001: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
# 2024-01-30 11:17:53.210606: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
# 2024-01-30 11:17:53.339576: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
# To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
# 2024-01-30 11:17:54.568146: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
# Tensorflow: 2.15.0
# /tmp/ipykernel_3814/2917868046.py:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
#   import kerastuner as kt
# kerastuner: 1.0.5
# keras_tuner: 1.3.5
# Python: 3.10.12
# numpy: 1.24.3
# pandas: 2.1.4
# sklearn version: 1.2.2
# sklearn path: ['/home/michaelye22/.local/lib/python3.10/site-packages/sklearn']
# matplotlib: 3.8.2
# seaborn: 0.13.0



Tensorflow: 2.15.0
kerastuner: 1.0.5
keras_tuner: 1.3.5
Python: 3.10.12
numpy: 1.24.3
pandas: 2.1.4
sklearn version: 1.2.2
sklearn path: ['/home/michaelye22/.local/lib/python3.10/site-packages/sklearn']
matplotlib: 3.8.2
seaborn: 0.13.0


## Set Global random seed to make sure we can replicate any model that we create (no randomness)

In [20]:
import random
import tensorflow as tf
import numpy as np
import os



np.random.seed(42)
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

os.environ['TF_DETERMINISTIC_OPS'] = '1'

# Data Preprocessing:

### Loading the training and testing data

train_values are the features (X), and train_labels is the target/label (Y)

In [21]:
train_X = pd.read_csv("train_features.csv")
train_y = pd.read_csv("train_labels.csv")

test_values = pd.read_csv("test_features.csv")

# print("train labels:\n", train_Y.head())

# print("train values:\n", train_X.head())
      
# print("test_values:\n", test_values.head())

### Check to see if there are any missing values in the data. If so, we have to do imputation

In [22]:
missing_train_X = train_X.isnull().sum().sum()
print("Number of missing values in train_X:", missing_train_X)

missing_train_y = train_y.isnull().sum().sum()
print("Number of missing values in train_Y:", missing_train_y)

missing_test_values = test_values.isnull().sum().sum()
print("Number of missing values in test_values:", missing_test_values )

Number of missing values in train_X: 0
Number of missing values in train_Y: 0
Number of missing values in test_values: 0


Since we have 0 missing values in each dataframe, we don't have to do imputation!

## Stratified train_test split

In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load the datasets

train_features = train_X
train_labels = train_y

# Merge the datasets on 'id'
merged_data = pd.merge(train_features, train_labels, on='id')

# Identifying the animal present in each image and creating a combined category
animal_columns = ['antelope_duiker', 'bird', 'blank', 'civet_genet', 'hog', 'leopard', 'monkey_prosimian', 'rodent']
merged_data['animal'] = merged_data[animal_columns].idxmax(axis=1)
merged_data['site_animal'] = merged_data['site'] + '_' + merged_data['animal']

# Checking the number of instances for each site_animal combination
combination_counts = merged_data['site_animal'].value_counts()
rare_combinations = combination_counts[combination_counts < 5]

# Separating the dataset into common and rare combinations
common_combinations = merged_data[~merged_data['site_animal'].isin(rare_combinations.index)]
rare_combinations_data = merged_data[merged_data['site_animal'].isin(rare_combinations.index)]

# Stratified split for common combinations
common_train_set, common_test_set = train_test_split(
    common_combinations, test_size=0.25, stratify=common_combinations['site_animal'], random_state=42)

# Randomly splitting rare combinations
total_samples = rare_combinations_data.shape[0]
train_samples = int(np.round(total_samples * 0.75))
rare_train_set = rare_combinations_data.sample(n=train_samples, random_state=42)
rare_test_set = rare_combinations_data.drop(rare_train_set.index)

# Combining the splits into final train and test sets
final_train_set = pd.concat([common_train_set, rare_train_set])
final_test_set = pd.concat([common_test_set, rare_test_set])

# Optional: Verifying the final distribution (can be commented out for large datasets)
final_train_distribution = final_train_set['site_animal'].value_counts(normalize=True)
final_test_distribution = final_test_set['site_animal'].value_counts(normalize=True)
final_distribution_summary = pd.DataFrame({
    'Train Distribution': final_train_distribution,
    'Test Distribution': final_test_distribution
})
print(final_distribution_summary.head())


                        Train Distribution  Test Distribution
site_animal                                                  
S0001_bird                        0.000647           0.000728
S0001_blank                       0.000404           0.000243
S0001_leopard                     0.003073           0.003153
S0001_monkey_prosimian            0.001051           0.000970
S0002_bird                        0.000566           0.000485


## Verify the 75/25 train_test split

In [24]:
# Calculate the number of samples in each set
num_train_samples = final_train_set.shape[0]
num_test_samples = final_test_set.shape[0]
total_samples = num_train_samples + num_test_samples

# Calculate the proportions
train_proportion = num_train_samples / total_samples
test_proportion = num_test_samples / total_samples

# Print out the proportions
print("Training Set Proportion: {:.2%}".format(train_proportion))
print("Test Set Proportion: {:.2%}".format(test_proportion))


Training Set Proportion: 74.99%
Test Set Proportion: 25.01%


## Making sure that train and test set have a 75/25 split for each site

In other words, for each site, around 75% should be in the training data and around 25% should be in the test data. This ensures niether the train or test set have a unbalanced amount of a certain site, leading to bias and bad predictions

In [25]:
# Calculate the count of each site in both sets
site_counts_train = final_train_set['site'].value_counts()
site_counts_test = final_test_set['site'].value_counts()

# Combine the counts into a single DataFrame for comparison
combined_site_counts = pd.DataFrame({'Train Count': site_counts_train, 'Test Count': site_counts_test})

# Calculate the total counts for each site
combined_site_counts['Total Count'] = combined_site_counts['Train Count'] + combined_site_counts['Test Count']

# Calculate the percentage split for each site
combined_site_counts['Train Percentage'] = (combined_site_counts['Train Count'] / combined_site_counts['Total Count']) * 100
combined_site_counts['Test Percentage'] = (combined_site_counts['Test Count'] / combined_site_counts['Total Count']) * 100

# Display the combined counts with percentage split
#print(combined_site_counts.head())
combined_site_counts.to_csv("site_percentage_check.csv")

# Check to see if there any rows of data with a train percentage below 70 or above 80 (the ideal is 75)
filtered_df = combined_site_counts[(combined_site_counts['Train Percentage'] < 70) | (combined_site_counts['Train Percentage'] > 80)]
#print(filtered_df)
#print(len(filtered_df)) # There are only 13 sites which have a bad train/test split, but they all side more towards the train set, which is good


#print(len(final_train_set))


There are only 13 sites which have a bad train/test split, but they all side more towards the train set, which is good

## Making sure the trian and test set have the 75/25 split for each animal 

In other words, for each animal, around 75% should be in the training data and around 25% should be in the test data. This ensures neither the train or test set have a unbalanced amount of a certain animal, leading to bias and bad predictions

In [26]:
# List of label columns
label_columns = ['antelope_duiker', 'bird', 'blank', 'civet_genet', 'hog', 'leopard', 'monkey_prosimian', 'rodent']

# Calculate the count of each label in both sets
label_counts_train = final_train_set[label_columns].sum()
label_counts_test = final_test_set[label_columns].sum()

# Combine the counts into a single DataFrame for comparison
combined_label_counts = pd.DataFrame({'Train Count': label_counts_train, 'Test Count': label_counts_test})

# Calculate the total counts for each label
combined_label_counts['Total Count'] = combined_label_counts['Train Count'] + combined_label_counts['Test Count']

# Calculate the percentage split for each label
combined_label_counts['Train Percentage'] = (combined_label_counts['Train Count'] / combined_label_counts['Total Count']) * 100
combined_label_counts['Test Percentage'] = (combined_label_counts['Test Count'] / combined_label_counts['Total Count']) * 100

# Display the combined counts with percentage split
#print(combined_label_counts)


Since each class has around a 75/25 split for train and test split, my data looks good!

## Modifying the data to include only the original columns

In [27]:
#print(final_train_set)

# Remove all the new features that I created for the stratified train_test split
train_X = final_train_set[['id', 'filepath', 'site']]
train_Y = final_train_set[['id','antelope_duiker','bird','blank','civet_genet','hog','leopard','monkey_prosimian','rodent']]

test_X = final_test_set[['id', 'filepath', 'site']]
test_Y = final_test_set[['id','antelope_duiker','bird','blank','civet_genet','hog','leopard','monkey_prosimian','rodent']]


# Make "id" the index column
train_X.set_index('id', inplace=True)
train_Y.set_index('id', inplace=True)

test_X.set_index('id', inplace=True)
test_Y.set_index('id', inplace=True)

#print(test_Y)


#print(train_X)
#print(train_Y)


## Make it so that if there already exists a dataset for train_X, train_Y, and test_X, then we will use those (so that each of my models are trained on the same data, making them deterministic)

In [28]:
import os

if not os.path.exists('train_split_X.csv'):
    print("create new directory")
    train_X.to_csv('train_split_X.csv')
    train_Y.to_csv('train_split_Y.csv')
    test_X.to_csv('test_split_X.csv')
    test_Y.to_csv('test_split_Y.csv')

else:
    print("used old directory")
    train_X = pd.read_csv('train_X.csv')
    train_Y = pd.read_csv('train_Y.csv')
    test_X = pd.read_csv('test_X.csv')
    test_Y = pd.read_csv("test_split.csv")

create new directory


## Check to see if each image has the same dimensions since that's important for data preprocessing

In [29]:
# from PIL import Image
# import os

# def check_image_dimensions(directory):
#     image_sizes = {}
#     for img_name in os.listdir(directory):
#         if not img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.tif')): # now also checks for .tif
#             continue
#         img_path = os.path.join(directory, img_name)
#         with Image.open(img_path) as img:
#             # Get image size
#             size = img.size
#             if size in image_sizes:
#                 image_sizes[size] += 1
#             else:
#                 image_sizes[size] = 1

#     for size, count in image_sizes.items():
#         print(f"For the {directory} directory, {count} images are of dimension {size}")

# # Use it on the train and test data only if this code segment was never ran in this coding session:

# # Use it on the train and test data:
# check_image_dimensions('train_features')
# check_image_dimensions('test_features')


# # # For the train_features directory, different dimensions found: {(160, 120), (960, 515), (640, 335), (960, 540), (640, 360), (360, 215), (160, 95), (360, 240)}
# # # For the test_features directory, different dimensions found: {(960, 515), (160, 120), (640, 335), (960, 540), (320, 215), (640, 360), (360, 240), (320, 240)}


## Resizing all the images to (640, 360)

I know the main sizes are (960, 540) and (640, 360). LUCKILY, they both close to a 16:9 aspect ratio, so both are viable options. However, it is generally better to shrink than to enlarge, so I'm choosing (640, 360).

Another thing to possibly try is to pick an even smaller 16:9 aspect ratio so that there is little to none upsampling (making images larger) 


In [30]:
# I will actually make the resizing function when I use the ImageDataGenertor. 
# By calling the resize function in the ImageDataGenerator, I won't have to save my images in my local folder and waste space. 

### Use Keras ImageDataGenerator() on Train/Test split
The ImageDataGenerator not only helps you load images from the disk but also allows you to perform **data augmentation**, which is a technique to increase the diversity of your training set by applying random transformations (like rotation, zoom, flips, etc.) to the images. This is very useful to prevent overfitting and helps the model generalize better.

In [31]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import numpy as np

def custom_resize(image, target_size=(640, 360)):
    # Convert the input NumPy array image to a PIL Image
    img = Image.fromarray(image)

    # Resize the image
    img_resized = img.resize(target_size, Image.ANTIALIAS)

    # Convert back to NumPy array and return
    return np.array(img_resized)





# Creating an instance of the ImageDataGenerator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Rescale the image pixel values to [0,1]
    preprocessing_function = custom_resize  # call the crop function on each image

    # Potential data augmentation techniques that won't affect the 32x32px center
    #brightness_range=[0.8, 1.2], 
    #channel_shift_range=20, 

    # I removed these transformations for the data augmentation since this project involves detecting tumor tissue in the center 32x32px region so I can't be doing zooming and other transformations for this project specifically

    # rotation_range=40,  # Random rotations
    # width_shift_range=0.2,  # Random horizontal shifts
    # height_shift_range=0.2,  # Random vertical shifts
    # shear_range=0.2,  # Shear transformations
    # zoom_range=0.2,  # Random zoom
    # horizontal_flip=True,  # Random horizontal flips
    # fill_mode='nearest'  # Strategy for filling in new pixels
)

val_datagen = ImageDataGenerator(rescale = 1./255, preprocessing_function = custom_resize)  # call the crop function on each image
test_datagen = ImageDataGenerator(rescale = 1./255, preprocessing_function = custom_resize) # call the crop function on each image



# Flow from dataframe method to load images using the dataframe
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df, # Use the training dataframe (with labels, id, and paths)
    x_col='path',
    y_col='label',
    target_size=(32, 32),  # The dimensions to which all images found will be resized. Change this as needed
    color_mode='rgb',
    class_mode='binary', # means that the labels are binary labels
    batch_size=32,
    shuffle=True, # This might introduce randomness if set to true, but if it's false, the it might lead to overfitting. So it's best to just save the neural network to ensure no randomness
    seed=42
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df, # Use the validation dataframe (with labels, id, and paths)
    x_col='path',
    y_col='label',
    target_size=(32, 32),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=False,
    seed=42
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df, # Use the testing dataframe (with labels, id, and paths)
    x_col='path',
    y_col='label',
    target_size=(32, 32),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=False,
    seed=42
)



# After setting this up, you can use train_generator as the input to the fit or fit_generator method of your Keras model, 
# which will load images in batches and train your model on them.





NameError: name 'train_df' is not defined

# Model Development

## Another thing to possibly try is to pick an even smaller 16:9 aspect ratio so that there is little to none upsampling (making images larger) 